# FE 630 Final

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from pandas_datareader import data as pdr 
import yfinance as yf 
from cvxopt import matrix, solvers 
from datetime import datetime, date 
from datetime import timedelta 
yf.pdr_override() # <== that's all it takes :-) from scipy import stats

from sklearn.preprocessing import PolynomialFeatures

from scipy import stats

## FF history data
Freely available for download from Ken French's website for the factors historical values (http://mba.tuck.dartmouth.edu/pages/faculty/ken.french)


In [ ]:
FF_data = pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
FF_data.columns =['date', 'Mkt-RF', 'SMB', 'HML', 'RF']
FF_data[['Mkt-RF', 'SMB', 'HML', 'RF']] = np.array(FF_data[['Mkt-RF', 'SMB', 'HML', 'RF']])/100 ## the original data is percentage, so need to divided by 100.

## Data Universe

1. CurrencyShares Euro Trust (FXE)
2. iShares MSCI Japan Index (EWJ)
3. SPDR GOLD Trust (GLD)
4. Powershares NASDAQ-100 Trust (QQQ)
5. SPDR S&P 500 (SPY)
6. iShares Lehman Short Treasury Bond (SHV)
7. PowerShares DB Agriculture Fund (DBA)
8. United States Oil Fund LP (USO)
9. SPDR S&P Biotech (XBI)
10. iShares S&P Latin America 40 Index (ILF)
11. iShares MSCI Pacific ex-Japan Index Fund (EPP)
12. SPDR DJ Euro Stoxx 50 (FEZ)

In [ ]:
daily_price_total = pdr.get_data_yahoo("FXE,EWJ,GLD,QQQ,SPY,SHV,DBA,USO,XBI,ILF,EPP,FEZ", start="2006-01-01", end="2020-06-20", interval = '1d').dropna()['Adj Close']
daily_return_total = daily_price_total.pct_change(1).dropna()
FF_total = FF_data[(FF_data['date']>=20070112) & (FF_data['date']<=20200620)]
FF_total.index = daily_return_total.index
daily_return_total[['Mkt-RF',	'SMB',	'HML',	'RF']] = FF_total[['Mkt-RF',	'SMB',	'HML',	'RF']]

## F-F Model

$\rho_i=r_f + \beta_i^3(\rho_M - r_f)+ b_i^s \rho_{SMB}+b_i^v\rho_{HML} + \alpha_i$

where, 
*    $\rho_i$ is the expected return of the stock,
*    $r_f$ is the risk-free return,
*    $\rho_{SMB}$ is Size Factor,
*    $\rho_{HML}$ is Value Factor.

And estimate $\beta_i^3$, $b_i^s$ and $b_i^v$ by linear regression.

So we need to change the function to:

$(\rho_i - r_f) = \beta_i^3(\rho_M - r_f)+ b_i^s \rho_{SMB}+b_i^v\rho_{HML} + \alpha_i$



In [ ]:
class FF_model():

  def __init__(self, lookback_period, date, stock_name):
    ''' lookback period < 40 days, short'''
    ''' lookback period > 40 and < 80 days, median '''
    ''' lookback period > 80 weeks, long '''
    ''' date is the day looking back '''
    ''' stock_names including DBA	EPP	EWJ	FEZ	FXE	GLD	ILF	QQQ	SHV	SPY	USO	XBI '''
    
    self.lp = lookback_period
    self.date = date
    self.name = stock_name

  def linear_model(self):

    period_data =  daily_return_total[daily_return_total.index < self.date][-self.lp:]
    rho_rf = (np.array(period_data[self.name]) - np.array(period_data['RF'])).reshape(len(period_data),1)
    rm_rf = np.array(period_data['Mkt-RF']).reshape(len(period_data),1)
    smb = (np.array(period_data.SMB)).reshape(len(period_data),1)
    hml = (np.array(period_data.HML)).reshape(len(period_data),1)

    X = np.hstack((rm_rf, smb, hml))
    y = rho_rf

    lr = LinearRegression()
    lr.fit(X,y)
    return lr.intercept_, lr.coef_

  def cal_beta(self):
    beta3 = self.linear_model()[1][0][0]
    if beta3 >= 0: 
      return beta3**(1/3)
    if beta3 < 0:
      return -(-beta3)**(1/3)
  
  def cal_alpha(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-1:]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X) 
    return float(alpha)

  def cal_alpha_2(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-2:-1]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X)
    return float(alpha) 

  def cal_alpha_3(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-3:-2]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X)
    return float(alpha)

  def cal_alpha_4(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-4:-3]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X)
    return float(alpha)

  def cal_alpha_5(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-5:-4]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X)
    return float(alpha)  

  def cal_alpha_6(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-6:-5]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X)
    return float(alpha) 

  def cal_alpha_300(self):
    alpha_total = np.zeros(300)
    for i in range(300):
      last_data = daily_return_total[daily_return_total.index < self.date][-(i+2):-(i+1)]
      rho_rf = last_data[self.name] - last_data['RF']
      rm_rf = last_data['Mkt-RF']
      smb = last_data['SMB']
      hml = last_data['HML']

      X = np.array([rm_rf, smb, hml])
      y = rho_rf
      alpha = y - np.sum(self.linear_model()[1][0]*X)
      alpha_total[i] = alpha

    return np.mean(alpha_total)

### Trend Estimator

In [ ]:
class trend_estimator():
    def __init__(self, lookback_period, date):
      ''' lookback period < 40 days, short'''
      ''' lookback period > 40 and < 80 days, median '''
      ''' lookback period > 80 weeks, long '''
      ''' date is the day looking back '''

      self.lp = lookback_period
      self.date = date


    def hml_estimate(self):
      period_data = daily_return_total[daily_return_total.index < self.date][-self.lp:]
      hml = np.array(period_data.HML)

      X = np.array(range(self.lp)).reshape(len(period_data),1)
      y = (np.cumprod(hml + 1) - 1).reshape(len(period_data),1)


      X_pre = np.array(self.lp).reshape(1, -1)

      quadratic_featurizer = PolynomialFeatures(degree=2)
      X_quadratic = quadratic_featurizer.fit_transform(X)
      X_quadratic_pre = quadratic_featurizer.transform(X_pre)
      regressor_quadratic = LinearRegression()
      regressor_quadratic.fit(X_quadratic, y)
      y_pre = regressor_quadratic.predict(X_quadratic_pre)
      
      return (1 + y_pre)**(1/self.lp) -1

    def smb_estimate(self):
      period_data = daily_return_total[daily_return_total.index < self.date][-self.lp:]
      smb = np.array(period_data.SMB)

      X = np.array(range(self.lp)).reshape(len(period_data),1)
      y = (np.cumprod(smb + 1) - 1).reshape(len(period_data),1)


      X_pre = np.array(self.lp).reshape(1, -1)

      quadratic_featurizer = PolynomialFeatures(degree=2)
      X_quadratic = quadratic_featurizer.fit_transform(X)
      X_quadratic_pre = quadratic_featurizer.transform(X_pre)
      regressor_quadratic = LinearRegression()
      regressor_quadratic.fit(X_quadratic, y)
      y_pre = regressor_quadratic.predict(X_quadratic_pre)
      
      return (1 + y_pre)**(1/self.lp) -1

    def mkt_estimate(self):
      period_data = daily_return_total[daily_return_total.index < self.date][-self.lp:]
      mkt = np.array(period_data['Mkt-RF'])

      X = np.array(range(self.lp)).reshape(len(period_data),1)
      y = (np.cumprod(mkt + 1) - 1).reshape(len(period_data),1)


      X_pre = np.array(self.lp).reshape(1, -1)

      quadratic_featurizer = PolynomialFeatures(degree=2)
      X_quadratic = quadratic_featurizer.fit_transform(X)
      X_quadratic_pre = quadratic_featurizer.transform(X_pre)
      regressor_quadratic = LinearRegression()
      regressor_quadratic.fit(X_quadratic, y)
      y_pre = regressor_quadratic.predict(X_quadratic_pre)
      
      return (1 + y_pre)**(1/self.lp) -1

## CAPM Model
$\rho_i=r_f + \beta_i^M(\rho_M - r_f) + \alpha_i$

In [ ]:
class CAPM_model():

  def __init__(self, lookback_period, date, stock_name):
    ''' lookback period < 40 days, short'''
    ''' lookback period > 40 and < 80 days, median '''
    ''' lookback period > 80 weeks, long '''
    ''' date is the day looking back '''
    ''' stock_names including DBA	EPP	EWJ	FEZ	FXE	GLD	ILF	QQQ	SHV	SPY	USO	XBI '''
    
    self.lp = lookback_period
    self.date = date
    self.name = stock_name

  def cal_beta(self):

    period_data =  daily_return_total[daily_return_total.index < self.date][-self.lp:]
    rho = np.array(period_data[self.name])
    rm = np.array(period_data['Mkt-RF'])+np.array(period_data['RF'])

    return (np.cov(rho,rm)/(np.var(rm)))[0][1]

## Investment Strategy

We will consider an portfolio optimization problem of the form:

$
\begin{cases} max \,\,\,\rho^T \omega - \lambda(\omega-\omega_p)^T \Sigma (\omega-\omega_p), &  \\ \sum_{i=1}^n \beta_i^m \omega_i = \beta_T^m, & \\ \sum_{i=1}^n \omega_i = 1, & -2 \leq \omega_i \leq 2 \end{cases}
$

Where,
*   $\Sigma$ is the the covariance matrix between the securities returns (computed from the Factor Model),
*   $\omega_p$ is the composition of a reference Portfolio (the previous
Portfolio when rebalancing the portfolio,
*   $\omega_p$ has all its components equal to $1/n$ for the first allocation,
*   $\lambda$ is a small regularization parameter to limit the turnover.
*   $\beta_i^m = \frac{cov(r_i,r_M)}{\sigma^2(r_M)}$ is the Beta of security $S_i$ as defined in the CAPM Model so that $\beta_i^m =\sum_{i=1}^n \beta_i^m \omega_i$ is the Beta of the Portfolio;
*   $\beta_T^m$ is the Portfolio's Target Beta, for example $\beta_T^m =-1,-0.5,0,0.5,1,1.5$.






### Theoretical Derivation of $\Sigma$

According to FF-model:
$\rho_i=r_f + \beta_i^3(\rho_M - r_f)+ b_i^s \rho_{SMB}+b_i^v\rho_{HML} + \alpha_i$

Thus, $cov(\rho_i, \rho_j)$

$=\beta_i^3 \beta_j^3 Var(\rho_M - r_f) + b_i^s b_j^s Var(\rho_{SMB}) + b_i^v b_j^v Var(\rho_{HML})$

$+ (\beta_i^3 b_j^s + \beta_j^3 b_i^s) cov(\rho_M-r_f, \rho_{SMB}) + (\beta_i^3 b_j^v+ \beta_j^3 b_i^v) cov(\rho_M-r_f, \rho_{HML})$

$+ (b_i^sb_j^v + b_j^s b_i^v)cov(\rho_{SMB},\rho_{HML})$

In [ ]:
## Based on the formula, functions should be added into class 
## to calculate the covariance between different variables

class FF_model():

  def __init__(self, lookback_period, date, stock_name):
      ''' lookback period < 40 days, short'''
      ''' lookback period > 40 and < 80 days, median '''
      ''' lookback period > 80 weeks, long '''
      ''' date is the day looking back '''
      ''' stock_names including DBA	EPP	EWJ	FEZ	FXE	GLD	ILF	QQQ	SHV	SPY	USO	XBI '''
    
      self.lp = lookback_period
      self.date = date
      self.name = stock_name

  def hml_estimate(self):
      period_data = daily_return_total[daily_return_total.index < self.date][-self.lp:]
      hml = np.array(period_data.HML)

      X = np.array(range(self.lp)).reshape(len(period_data),1)
      y = (np.cumprod(hml + 1) - 1).reshape(len(period_data),1)


      X_pre = np.array(self.lp).reshape(1, -1)

      quadratic_featurizer = PolynomialFeatures(degree=2)
      X_quadratic = quadratic_featurizer.fit_transform(X)
      X_quadratic_pre = quadratic_featurizer.transform(X_pre)
      regressor_quadratic = LinearRegression()
      regressor_quadratic.fit(X_quadratic, y)
      y_pre = regressor_quadratic.predict(X_quadratic_pre)
      
      return (1 + y_pre)**(1/self.lp) -1

  def smb_estimate(self):
      period_data = daily_return_total[daily_return_total.index < self.date][-self.lp:]
      smb = np.array(period_data.SMB)

      X = np.array(range(self.lp)).reshape(len(period_data),1)
      y = (np.cumprod(smb + 1) - 1).reshape(len(period_data),1)


      X_pre = np.array(self.lp).reshape(1, -1)

      quadratic_featurizer = PolynomialFeatures(degree=2)
      X_quadratic = quadratic_featurizer.fit_transform(X)
      X_quadratic_pre = quadratic_featurizer.transform(X_pre)
      regressor_quadratic = LinearRegression()
      regressor_quadratic.fit(X_quadratic, y)
      y_pre = regressor_quadratic.predict(X_quadratic_pre)
      
      return (1 + y_pre)**(1/self.lp) -1

  def mkt_estimate(self):
      period_data = daily_return_total[daily_return_total.index < self.date][-self.lp:]
      mkt = np.array(period_data['Mkt-RF'])

      X = np.array(range(self.lp)).reshape(len(period_data),1)
      y = (np.cumprod(mkt + 1) - 1).reshape(len(period_data),1)


      X_pre = np.array(self.lp).reshape(1, -1)

      quadratic_featurizer = PolynomialFeatures(degree=2)
      X_quadratic = quadratic_featurizer.fit_transform(X)
      X_quadratic_pre = quadratic_featurizer.transform(X_pre)
      regressor_quadratic = LinearRegression()
      regressor_quadratic.fit(X_quadratic, y)
      y_pre = regressor_quadratic.predict(X_quadratic_pre)
      
      return (1 + y_pre)**(1/self.lp) -1 


  

  def linear_model(self):    
    period_data =  daily_return_total[daily_return_total.index < self.date][-self.lp:]
    rho_rf = (np.array(period_data[self.name]) - np.array(period_data['RF'])).reshape(len(period_data),1)
    rm_rf = (np.array(period_data['Mkt-RF']) - np.array(period_data['RF'])).reshape(len(period_data),1)
    smb = (np.array(period_data.SMB)).reshape(len(period_data),1)
    hml = (np.array(period_data.HML)).reshape(len(period_data),1)

    X = np.hstack((rm_rf, smb, hml))
    y = rho_rf

    lr = LinearRegression()
    lr.fit(X,y)
    return lr.intercept_, lr.coef_

  def expected_return(self):
    rm_rf = self.mkt_estimate()
    smb = self.smb_estimate()
    hml = self.hml_estimate()
    X = np.array([rm_rf, smb, hml])
    expected_return = np.sum(self.linear_model()[1][0]*X) + self.linear_model()[0]
    return expected_return

  def var_rhoM_rhoSMB_rhoHML(self):
    
    period_data =  daily_return_total[daily_return_total.index < self.date][-self.lp:]
    rm_rf = (np.array(period_data['Mkt-RF']) - np.array(period_data['RF']))
    smb = (np.array(period_data.SMB))
    hml = (np.array(period_data.HML))
    return np.var(rm_rf), np.var(smb), np.var(hml)

  def cov_MfSMB_MfHML_SMBHML(self):
    
    period_data =  daily_return_total[daily_return_total.index < self.date][-self.lp:]
    rm_rf = (np.array(period_data['Mkt-RF']) - np.array(period_data['RF']))
    smb = (np.array(period_data.SMB))
    hml = (np.array(period_data.HML))
    return np.cov(rm_rf, smb)[0][1], np.cov(rm_rf, hml)[0][1], np.cov(smb,hml)[0][1]
  
  def cal_cov(self, self2):
    beta3i, bsi, bvi = self.linear_model()[1][0]
    beta3j, bsj, bvj = self2.linear_model()[1][0]
    cof_vector = np.array([beta3i*beta3j, bsi*bsj, bvi*bvj, beta3i*bsj+beta3j*bsi,
                           beta3i*bvj+beta3j*bvi, bsi*bvj+bsj*bvi])
    var_vector = np.array(self.var_rhoM_rhoSMB_rhoHML())
    cov_vector = np.array(self.cov_MfSMB_MfHML_SMBHML())
    return np.sum(np.hstack((var_vector, cov_vector)) * cof_vector)

  def var(self):
    
    period_data =  daily_return_total[daily_return_total.index < self.date][-self.lp:]
    rho_rf = (np.array(period_data[self.name]) - np.array(period_data['RF']))
    return np.var(rho_rf)

  def cal_alpha(self):

    last_data = daily_return_total[daily_return_total.index < self.date][-1:]
    rho_rf = last_data[self.name] - last_data['RF']
    rm_rf = last_data['Mkt-RF']
    smb = last_data['SMB']
    hml = last_data['HML']

    X = np.array([rm_rf, smb, hml])
    y = rho_rf
    alpha = y - np.sum(self.linear_model()[1][0]*X) 
    return float(alpha)

### Implement of Strategy
$
\begin{cases} max \,\,\,\rho^T \omega - \lambda(\omega-\omega_p)^T \Sigma (\omega-\omega_p), &  \\ \sum_{i=1}^n \beta_i^m \omega_i = \beta_T^m, & \\ \sum_{i=1}^n \omega_i = 1, & -2 \leq \omega_i \leq 2 \end{cases}
$

Define $S_{60}^{90}(0.5)$ - say using 60 days for estimation of covariance, 90 days for estimation of Expected Returns and a target $\beta = 0.5$

*   The portfolios will be rebalanced once a week.
*   Divide the overall analysis period into 3 sub-periods: before, during and after the subprime crisis.



The formula showed before is not easily implemented. It needs to be transformed to another expression form:

let $\omega_c = \omega - \omega_p$

Then, the optimization problem will be transformed to:

$
\begin{cases} max \,\,\,\rho^T \omega_p + \rho^T\omega_c - \lambda\omega_c^T \Sigma \omega_c, &  \\ \sum_{i=1}^n \beta_i^m (\omega_{ci} + \omega_{pci}) = \beta_T^m, & \\ \sum_{i=1}^n \omega_{ci} = 0, & -2-\omega_{pci} \leq \omega_{ci} \leq 2 - \omega_{pci} \end{cases}
$

Because the $\rho^T\omega_p$ is a constant value, so the target function is same to $min\,\,\, \frac{1}{2}\omega_c^T (2\lambda\Sigma) \omega_c - \rho^T\omega_c$

In [ ]:
universe = ['DBA','EPP','EWJ','FEZ','FXE','GLD','ILF','QQQ','SHV','SPY','USO','XBI']
solvers.options['show_progress'] = False

class portfolio():

  def __init__(self,date,return_lookback_period,cov_lookback_period,target_beta,d_lambda):
    self.date = date
    self.rlp = return_lookback_period
    self.clp = cov_lookback_period
    self.tb = target_beta
    self.lbd = d_lambda
  
  def cov_matrix(self):
    ''' get the stocks covariance matrice based on cov_lookback_period'''
    ''' cov matrix is calculated by FF model'''

    global universe
    n = len(universe)
    A = np.zeros([n,n])
    for i in range(n):
      for j in range(n):
        stock1 = FF_model(self.clp,self.date,universe[i])
        stock2 = FF_model(self.clp,self.date,universe[j])
        A[i,j] = stock1.cal_cov(stock2)
    
    ''' Because there is a floating point error in '''
    ''' Python's judgment of whether the matrix is a semi-positive definite matrix,''' 
    ''' there will be a very small negative eigenvalue (actually 0), which will cause ''' 
    ''' the positive definite matrix to be misjudged as a non-positive definite matrix '''
    ''' our method is add a small number to the values on the diagonal '''    

    for i in range(n):
      A[i,i] = A[i,i] + 0.0000000000001
    return A

  def return_vector(self):
    ''' get the stocks expected return vector based on return_lookback_period '''
    ''' use alpha and cor_vector to predict '''

    global universe
    n = len(universe)
    return_vector = np.zeros(n)
    for i in range(n):
      stock_name = universe[i]
      estimator = FF_model(self.rlp, self.date, stock_name)
      return_vector[i] = estimator.expected_return()

    return return_vector

  def beta_vector(self):
    ''' get the beta of each stock based on cov_lookback_period '''

    global universe
    n = len(universe)
    v = np.zeros(n)
    for i in range(n):
      stock = CAPM_model(self.clp,self.date,universe[i])
      v[i] = stock.cal_beta()
    
    return v

  def opti_weights_wc(self, wpc):
    
    ## Definition
    cov_ma = self.cov_matrix()
    r_vector = self.return_vector()
    beta_vector = self.beta_vector()
    n = len(r_vector)

    ## optimization problem
    P = matrix(cov_ma*2*self.lbd)
    q = matrix(-r_vector)

    ## Inequality constraints
    ## two inequality constraints
    G = matrix(np.vstack((np.diag([1]*n), np.diag([-1]*n))).astype('double'))
    h = matrix(np.hstack((np.zeros(n)+0.5-wpc, np.zeros(n)+0.5+wpc)))

    ## Equality constraints
    ## two equality constraints
    A = matrix(np.vstack((self.beta_vector(), np.zeros(n)+1)))
    b = matrix(np.hstack((self.tb-np.sum(beta_vector*wpc),0)))

    result = solvers.qp(P,q,G,h,A,b)
    weights_c = np.array(result['x'])
    wc_vector = np.zeros(n)
    for i in range(n):
      wc_vector[i] = weights_c[i][0]

    return wc_vector
  
  def optimal_w(self, wpc):
    w_c = self.opti_weights_wc(wpc)
    return w_c + wpc
  
  def test_beta(self, wpc):
    weights = self.optimal_w(wpc)
    beta = np.sum(weights*self.beta_vector())
    return beta

## Back Test in different sub-periods
Different Strategies's performance before, during and after the subprime crisis.

*   One strategy has two dimensions: target beta($\beta_T^m$) and lambda($\lambda$).
*   The portfolio will be rebalanced every week based on the strategy.
*   The benchmark will be the Market Portfolio S&P 500 ( SPY ETF)
*   Assuming that you invest $\$100$ at the first allocation date in Portfolio and $\$100$ in the S&P 500 (when you benchmark your strategies against the Market, the SPY is representing the S&P500 Index).

*   The maximum lookback period is $180$ days($26$ weeks) to ensure sufficient data




In [ ]:
def get_date(start_date, add_days):

    return str(daily_return_total[daily_return_total.index > start_date].index[add_days-1])[:10]

def maximum_10_drawdown(array):
    max_dd = 0
    for i in range(len(array)-10):
      ten_day_array = array[i:i+10]
      max = np.max(ten_day_array)
      min = np.min(ten_day_array)
      ten_dd = (max-min)/min
      max_dd = np.max(np.array([ten_dd, max_dd]))
    return max_dd

def sample_para_VaR(data, size):

    sample = np.random.choice(data, size, replace=True)
    VaR_5 = np.percentile(sample, 5)
    return VaR_5

def insert_series(series):    
    new_series = [i for i in range(len(series))]
    n = len(series)
    steps = int(n/5)
    ranges = list(np.arange(0,len(series),step=steps))
    for i in range(len(new_series)):
      if i in ranges:
        new_series[i] = series[i]
      else:            
        new_series[i] = ''
        if i == n-1:
            new_series[i] = series[i]
    return new_series

class simulation():

    def __init__(self,date,duration,return_lookback_period,cov_lookback_period,target_beta,d_lambda):
        ''' date is the first allocation date '''
        ''' duration is the time holding portfolio '''
        
        self.date = date
        self.D = duration
        self.rlp = return_lookback_period
        self.clp = cov_lookback_period
        self.tb = target_beta
        self.lbd = d_lambda

    def cal_profit(self):
        daily_return = np.zeros(self.D)
        wpc = np.zeros(12) + 1/12
        n = 12
        d = self.date
        for i in range(self.D):
          ## rebalance the portfolio every week.
          if i % 6 == 0:

            date = get_date(d, i)
            p = portfolio(date,self.rlp,self.clp, self.tb, self.lbd)
            next_day_data =  daily_return_total[daily_return_total.index >= date].iloc[0,:][:12]
            try:
              weights = np.round(p.optimal_w(wpc),3)
            except:
              weights = np.array([0,0,0,0,0,0,0,0,0,1,0,0])
            wpc = weights
            daily_return[i] = np.sum(weights*np.array(next_day_data))
          else:
            date = get_date(d, i)
            next_day_data =  daily_return_total[daily_return_total.index >= date].iloc[0,:][:12]
            daily_return[i] = np.sum(weights*np.array(next_day_data))
          
        cul_return = np.cumprod(daily_return + 1) - 1
        return np.round(daily_return,6) ,np.round(cul_return,3)
    
    def benchmark(self):
    
        next_day_data =  daily_return_total[daily_return_total.index >= self.date]
        daily_return = np.array(next_day_data['SPY'][:self.D])
        risk_free_return = np.array(next_day_data['RF'][:self.D])
        cul_return = np.cumprod(daily_return + 1) - 1
        return daily_return, np.round(cul_return,3) , risk_free_return 

    def profit_plot(self):
        ''' plot the cumulated profit '''

        date_vector = [get_date(self.date, i+1)  for i in range(self.D)]
        po_trend = self.cal_profit()[1]
        po_vector = self.cal_profit()[0]
        benchmark_trend = self.benchmark()[1]
        benchmark_vector = self.benchmark()[0]
        risk_free_return = self.benchmark()[2]

        plt.style.use('ggplot')
        plt.plot(date_vector,po_trend,color='r',linewidth=1,alpha=0.6,label="portfolio")
        plt.plot(date_vector,benchmark_trend, color='b',linewidth=1,alpha=0.6,label='benchmark')
        plt.xticks(insert_series(date_vector))
        plt.legend(bbox_to_anchor=(0.3, 1), loc=1, borderaxespad=0.)
        plt.show()

        plt.hist(po_vector, edgecolor='k',bins = 40, alpha=0.35)
        plt.show()


        print('portfolio PnL is {0}'.format(po_trend[-1]*100))
        print('benchmark PnL is {0}'.format(benchmark_trend[-1]*100))

        print('portfolio cumulated return is {0}'.format(po_trend[-1]))
        print('benchmark cumulated return is {0}'.format(benchmark_trend[-1]))

        print('portfolio average return is {0}'.format(250*np.mean(po_vector)))
        print('benchmark average return is {0}'.format(250*np.mean(benchmark_vector)))
        
        print('portfolio standard deviation is {0}'.format(np.sqrt(250)*np.std(po_vector)))
        print('benchmark standard deviation is {0}'.format(np.sqrt(250)*np.std(benchmark_vector)))

        print('portfolio Sharpe ratio is {0}'.format((np.sqrt(250)*(np.mean(po_vector) - np.mean(risk_free_return))/np.std(po_vector))))
        print('benchmark Sharpe ratio is {0}'.format((np.sqrt(250)*(np.mean(benchmark_vector) -np.mean(risk_free_return))/np.std(benchmark_vector))))

        print('portfolio max 10 DD is {0}'.format(maximum_10_drawdown(po_trend + 1)))
        print('benchmark max 10 DD is {0}'.format(maximum_10_drawdown(benchmark_trend + 1)))

        print('portfolio skewness is {0}'.format(stats.skew(po_vector)))
        print('benchimark skewness is {0}'.format(stats.skew(benchmark_vector)))

        print('portfolio kurtosis is {0}'.format(stats.kurtosis(po_vector)))
        print('benchimark kurtosis is {0}'.format(stats.kurtosis(benchmark_vector)))

        print('portfolio VaR is {0}'.format(sample_para_VaR(po_vector, len(po_vector))))
        print('benchimark VaR is {0}'.format(sample_para_VaR(benchmark_vector, len(benchmark_vector))))


## Performance
The input structure is : (begin date, duration, return lookback period, covariance lookback period, target beta, lambda)

It will take 20 minutes for whole period simulation, 3 minutes for 2007-year simulation in colab.

### Before crisis

In [ ]:
## from 2007-04-02 to 2007-12-31
si = simulation('2007-04-01',190,60,60,1,0.00001)
si.profit_plot()

### During Crisis

In [ ]:
## from 2018-01-01 to 2018-12-31
si = simulation('2008-01-01',252,60,60,1,0.00001)
si.profit_plot()

### Whole period

In [ ]:
si = simulation('2007-04-01',3329,60,60,1,0.00001)
si.profit_plot()